In [18]:
#Import libraries
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from pandas import *

In [19]:
#Loading data
rfm = pd.read_csv('../data/rfm.csv', encoding = 'utf-8',index_col=None, sep = '\t')
rfm.head()

,CustomerID,frequency,monetary_value,recency,r,f,m,rfm,rfm_score,Segment
0,12820.0,59,942.34,3,1,2,2,122,1.67,potential loyalists
1,12821.0,6,92.72,214,4,4,4,444,4.00,hibernating
2,12822.0,46,948.88,70,3,2,2,322,2.33,at risk
3,12823.0,5,1759.50,74,3,4,1,341,2.67,hibernating
4,12824.0,25,397.12,59,3,3,3,333,3.00,hibernating


In [39]:
X = rfm.iloc[:, 1:4]
y = rfm.Segment
y.nunique()

8

In [21]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
label = le.fit_transform(y)

rfm['cluster'] = label 

rfm.sample(10)

y = rfm.cluster

In [41]:
#Train, val and test split
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.3)

X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.3)

In [42]:
#Standardization of our datas
from sklearn.preprocessing import StandardScaler
import pickle

# Create scaler object
scaler = StandardScaler() 
#Fit scalar object to our train set
scaler.fit(X_train)

#Save the model
#filename = 'standardization.pkl'
#pickle.dump(scaler, open(filename, 'wb'))

#Transform train, val and test into standardize datas
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

X_scal = pd.DataFrame(scaler.fit_transform(X))

/home/celiabayet/.local/lib/python2.7/site-packages/ipykernel_launcher.py:15: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  from ipykernel import kernelapp as app
/home/celiabayet/.local/lib/python2.7/site-packages/ipykernel_launcher.py:16: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  app.launch_new_instance()
/home/celiabayet/.local/lib/python2.7/site-packages/ipykernel_launcher.py:17: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


In [50]:
from sklearn.metrics import roc_curve, auc

n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200]
train_results = []
test_results = []
for estimator in n_estimators:
    rf = RandomForestClassifier(n_estimators=estimator, n_jobs=-1)
    rf.fit(X_train, y_train)
    train_pred = rf.predict(X_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = rf.predict(X_val)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_val, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)
    
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(n_estimators, train_results, 'b', label='Train AUC')
line2, = plt.plot(n_estimators, test_results, 'r', label='Val AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('n_estimators')
plt.show()

ValueError: multiclass format is not supported

In [43]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

#Using kfold to get 5 different training set
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5)
results = pd.DataFrame(columns=['training_score', 'test_score'])
fprs, tprs, scores = [], [], []

from sklearn import metrics

pred = []
accuracy = []
for (train, test), i in zip(cv.split(X_scal, y), range(5)):
    rf.fit(X_scal.iloc[train], y.iloc[train])
    y_test_pred = rf.predict(X_scal.iloc[test])
    pred.append(y_test_pred)
    accuracy.append(metrics.accuracy_score(y.iloc[test], y_test_pred))

print("Accuracy is {}".format(accuracy))

Accuracy is [1.0, 1.0, 0.9985795454545454, 1.0, 1.0]


In [24]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

#Fit to our data
rf.fit(X_train, y_train);

In [25]:
y_val_pred = rf.predict(X_val)
pd.crosstab(y_val, y_val_pred, rownames=['Actual Cluster'], colnames=['Predicted Cluster'])

Predicted Cluster,0,1,2,3,4,5,6,7
Actual Cluster,,,,,,,,
0,80,0,0,0,0,0,0,0
1,0,9,0,0,0,0,0,0
2,0,0,61,0,0,0,0,0
3,0,0,0,269,0,0,0,0
4,0,0,0,0,69,0,0,0
5,0,0,0,0,0,23,0,0
6,0,0,0,0,0,0,192,0
7,0,0,0,0,0,0,0,36


In [26]:
list(zip(X_train, rf.feature_importances_))

[(array([ 0.29944971,  0.23847376, -0.38788215,  0.34892521, -0.61867097,
         -0.6146091 ]), 0.2484964235373818),
 (array([ 0.61344818,  1.59092472, -0.60734977, -0.56765444, -0.61867097,
         -1.55289709]), 0.03482754011601664),
 (array([ 0.07267303, -0.67143154,  1.6372055 ,  1.26550485, -0.61867097,
          0.32367889]), 0.22825198936727273),
 (array([ 1.24144514,  2.10735843, -0.43776115,  0.34892521, -1.54988983,
         -1.55289709]), 0.22951872896236544),
 (array([-0.76465623,  0.33226353, -0.92657541, -1.48423409,  1.24376675,
         -0.6146091 ]), 0.2454404186709766),
 (array([ 1.31122257,  2.11791731, -0.95650282, -1.48423409, -1.54988983,
         -1.55289709]), 0.013464899345986428)]

In [38]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC(gamma="scale")
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(X_train, y_train)

y_val_pred = clf.predict(X_val)

print("Accuracy is {}".format(metrics.accuracy_score(y_val_pred, y_val)))

Accuracy is 1.0
